# Importing Libraries

In [1]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Importing Dataset

In [2]:
df = pd.read_csv('spam.csv',encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# Dropping Unnecessary Columns

In [3]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis='columns',inplace=True)
df.columns = ['label', 'message']
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Tokenization & Stemming

In [4]:
ps = PorterStemmer()

In [5]:
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower()
    review = nltk.word_tokenize(review)
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [6]:
corpus[0:5]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though']

# Implementing Bag Of Words

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_features = 5000)
X = cv.fit_transform(corpus).toarray()

# Create Countvectorizer Pickle File

In [9]:
pickle.dump(cv,open('transformer.pkl','wb'))

In [10]:
y = pd.get_dummies(df['label'])
y = y.iloc[:,1].values

In [11]:
# print(df['label'],"\n y = ",y)
# Ham = 0, Spam = 1

# Train Test Split

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=0)

# Naive Bayes Model

In [13]:
from sklearn.naive_bayes import MultinomialNB

In [14]:
spam_detect_model = MultinomialNB()
spam_detect_model.fit(X_train,y_train)

MultinomialNB()

In [15]:
y_pred = spam_detect_model.predict(X_test)

# Accuracy & Result

In [16]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
cm

array([[935,  14],
       [  7, 159]], dtype=int64)

In [17]:
acc_score = accuracy_score(y_test,y_pred)
acc_score

0.9811659192825112

# Creating Pickle File of the Trained Model

In [18]:
pickle.dump(spam_detect_model,open('model.pkl','wb'))